In [1]:
import cvxpy as cp

# 저자

- TA: 성준모 (Joonmo Sung)
- SYSTEMS MODELING AND PROGRAMMING LAB DEPARTMENT OF INDUSTRIAL ENGINEERING, YONSEI UNIVERSITY - SYMPLY
- 문의: `sjm21314@naver.com`,`sjm21314@yonsei.ac.kr`
***
본 강의자료는 Keha,A. B.,Khowala,K.,&Fowler,J. W. (2009). Mixed integer programming formulations for single machine scheduling problems. Computers & Industrial Engineering, 56(1), 357-367 논문을 바탕으로 제작되었으며,

반도체데이터사이언스 협동과정 수업 용도 이외의 목적으로 저자의 허락 없이 다른 사람들과 공유할 수 없습니다.

## Dataset
### Set
- N = $\{1,2,...,n\}, j \in N$
### Parameters
- $p_j$: processing time of job $j$
- $d_j$: due date of job $j$
- $w_j$: weight of job $j$
- $r_j$: release date of job $j$

In [2]:
# Offline 상황을 가정.

n = 5 # 작업의 개수

N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌
p = [4, 3, 7, 2, 2] # 가공시간 데이터를 리스트 자료구조를 이용해 만들어 줌
d = [5, 6, 8, 8, 17] # 납기 데이터를 리스트 자료구조를 이용해 만들어 줌
w = [5, 4, 3, 2, 1] # 가중치 데이터를 리스트 자료구조를 이용해 만들어 줌
r = [1, 2, 3, 3, 5] # 작업의 release date(도착 시점) 데이터를 리스트 자료구조를 이용해 만들어 줌

M = 100 # big M

In [3]:
obj_select = 'twt' # 'wct', 'tj', 'twt', 'L_max' 중 하나 선택
release_time = False

## 4. Assignment and positional date variables

### Decision Variables
- $u_{j,k}$: 작업 $j$가 position k에 있으면 1, 아니면 0인 이진변수, $u_{j,k} \in \{0, 1\} \ \forall j \in N, \forall k \in N$
- $\gamma_k$: 포지션 $k$에 있는 작업의 completion time

In [4]:
# Decision Variable 만들기
u = {(j, k): cp.Variable(boolean = True) for j in N for k in N}
gamma = {(j): cp.Variable(nonneg = True) for j in N}

### 기본 constraint
- $\sum_{k \in N}u_{j,k} = 1 \quad \forall j \in N$
- $\sum_{j \in N}u_{j,k} = 1 \quad \forall k \in N$
- $\gamma_j \geq \sum_{j \in N}p_ju_{j,1} $
- $\gamma_k \geq \gamma_{k-1} + \sum_{j \in N}p_ju_{j,k} \quad \forall k = 2,3,..N$


In [6]:
# 제약식 리스트 선언
constraints = []

# 제약식 추가
constraints += [sum(u[j,k] for k in N) == 1 for j in N]
constraints += [sum(u[j,k] for j in N) == 1 for k in N]
constraints += [gamma[0] >= sum(p[j]*u[j,0] for j in N)]
constraints += [gamma[k] >= gamma[k-1] + sum(p[j]*u[j,k] for j in N) for k in N[1:]] # N[1:] 은 1번 index(2번째)부터 시작함을 의미

#### Release time 반영 제약식
$\gamma_k \geq \sum_{j \in N}(p_j + r_j)u_{j,k} \quad \forall k \in N$

In [7]:
if release_time == True:
    constraints += [gamma[k] >= sum((p[j] + r[j]) * u[j,k] for j in N) for k in N]

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{k=1}^n U_k$

$ \\ \gamma_k \leq \sum_{j \in N}d_ju_{j,k} + MU_k \quad \forall k \in N$

In [8]:
if obj_select == "tj":
    U = {(j): cp.Variable(boolean = True) for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    # Tardy job 제약식
    constraints += [gamma[k] <= sum(d[j]*u[j,k] for j in N) + M*U[k] for k in N]

    # 목적식
    obj = cp.Minimize(sum(U[j] for j in N))

#### To minimize $L_{max}$
$ Minimize \ L_{max}$

$ \\ s.t. \  L_{max} \geq \gamma_k - \sum_{j \in N} d_ju_{j,k} \quad \forall k \in N$

In [9]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = cp.Variable(nonneg = True) # L_max를 나타내는 연속형 변수

    # Lmax 제약식
    constraints += [L_max >= gamma[k] - sum(d[j]*u[j,k] for j in N) for k in N]

    # 목적식
    obj = cp.Minimize(L_max)

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n w_j C_j$

$\\ C_j \geq \gamma_k - M(1-u_{j,k}) \quad \forall j,k \in N$

$\\ C_j \geq 0 \quad \forall j \in N$

In [10]:
if obj_select == "wct":
    # 종료 시점을 나타내는 변수 추가
    C = {(j): cp.Variable(nonneg = True) for j in N}

    # C_j 제약식
    constraints += [C[j] >= gamma[k] - M*(1-u[j,k]) for j in N for k in N]

    # 목적식
    obj = cp.Minimize(sum(w[j]*C[j] for j in N))

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$
$\\ C_j \geq \gamma_k - M(1-u_{j,k}) \quad \forall j,k \in N$

$\\ C_j \geq 0 \quad \forall j \in N$

$\\ T_j \geq C_j - d_j \quad \forall j \in N$

$\\ T_j \geq 0 $

In [11]:
if obj_select == "twt":
    # 종료 시점을 나타내는 변수 추가
    C = {(j): cp.Variable(nonneg = True) for j in N}
    # Tardiness를 나타내는 변수 추가
    T = {(j): cp.Variable(nonneg = True) for j in N}

    # 추가 제약식
    constraints += [C[j] >= gamma[k] - M*(1-u[j,k]) for j in N for k in N]
    constraints += [T[j] >= C[j]-d[j] for j in N]

    # 목적식
    obj = cp.Minimize(sum(w[j]*T[j] for j in N))

In [12]:
prob = cp.Problem(obj, constraints)
prob.solve()

np.float64(31.0)

In [13]:
print("최적해 상태:", prob.status)
print("최적값:", prob.value)

최적해 상태: optimal
최적값: 31.0


In [14]:
for (j,k), dv in u.items():
    if dv.value > 1e-6:
        print(f"u_{j+1}_{k+1}: {dv.value}")

u_1_1: 1.0
u_2_2: 1.0
u_3_4: 1.0
u_4_3: 1.0
u_5_5: 1.0


In [15]:
for j, dv in gamma.items():
    if dv.value > 1e-6:
        print(f"gamma_{j+1}: {dv.value}")

gamma_1: 4.0
gamma_2: 7.0
gamma_3: 9.0
gamma_4: 16.0
gamma_5: 18.0
